<b>Вариант 15</b>

Особь – битовая последовательность размера n (кол-во грузов)

Начальная популяция – кол-во особей всегда = 200


<p><b>Начальная популяция:</b></p>
<p>1.1 случайная генерация</p>
<p><b>Отбор:</b></p>
<p>2.2 выбрать только 20% самых приспособленных особей</p>
<p><b>Скрещивание:</b></p>
<p>3.2 однородный (каждый бит от случайно выбранного родителя)</p>
<p><b>Мутация:</b></p>
<p>4.3 добавление 1 случайной вещи 5% особей</p>
<p><b>Новая популяция:</b></p>
<p>5.2 «штраф» за «старость» - 10% функции приспособленности, выбор лучших</p>
<p><b>Оценка результата:</b></p>
<p>Наступила сходимость (функция приспособленности лучшей особи в популяциях отличается не более, чем на стоимость самой дешевой вещи) или прошло 500 поколений</p>

In [1]:
import numpy as np
import pandas as pd
import random
from random import randint

In [2]:
df = pd.read_csv('12.txt', delimiter=' ', engine='python')
parameters = df.columns.tolist()
df.columns = ['v', 'c']
df['w'] = df.index
data = df[['w','v','c']].values.tolist()
data = pd.DataFrame.from_records(data, columns=['w','v','c'])

data

,w,v,c
0,796.0,1.0,158.0
1,310.0,0.8,267.0
2,980.0,0.8,151.0
3,439.0,0.7,176.0
4,1338.0,1.0,136.0
5,694.0,1.1,104.0
6,595.0,0.7,316.0
7,1310.0,0.6,352.0
8,1563.0,0.6,161.0
9,525.0,0.6,126.0


In [3]:
# Const

initialPopulation = 200 #Начальная популяция
numGenes = len(data) #Количество генов
W = int(parameters[0]) #Грузоподъемность
V = int(parameters[1]) #Вместимость
mutationInd = 0.05
selectionInd = 0.2
cInd = 0.9

In [4]:
# Проверка ограничений

def checkW(individual):
    sumW = 0
    for i in range (0, numGenes):
        sumW += data.at[i,'w'] * individual[i]
    if sumW > W:
        return 0
    else: 
        return 1

def checkV(individual):
    sumV = 0
    for i in range (0, numGenes):
        sumV += data.at[i,'v'] * individual[i]
    if sumV > V:
        return 0
    else: 
        return 1

In [5]:
# Начальная популяция: случайная генерация

#Создание битовой особи
def createIndividual():
    individual = [randint(0, 1) for x in range(numGenes)]
    while not (checkW(individual) and checkV(individual)):
        individual = [randint(0, 1) for x in range(numGenes)]
    return individual

#Создание начальной популяции
def createInitialPopulation():
    return [createIndividual() for _ in range(0, initialPopulation)]

#Ценность каждой особи
def individualC(individual):
    sumC = 0
    for i in range(0, numGenes):
        sumC += data.at[i,'c'] * individual[i]
    return sumC

In [6]:
# Отбор: 20% самых приспособленных особей

def selection(population):
    population.sort(key=lambda x: individualC(x), reverse=True)
    newPopulation = []
    for i in range (0, round(len(population) * selectionInd)):
            newPopulation.append(population[i])
    return newPopulation

In [7]:
# Скрещивание: однородный (каждый бит от случайно выбранного родителя)
# Каждая особь скрещивается 1 раз за 1 поколение, 1 пара дает 2 потомка

def newChild(mother, father):
    child = []
    mask = createIndividual()
    for i in range(0, len(mask)):
        if mask[i] == 1:
            child.append(mother[i])
        else:
            child.append(father[i])
    return child

def crossingover(population):
    childs = []
    while len(population) > 0:
        random.shuffle(population)
        mother = population.pop()
        father = population.pop()
        child1 = newChild(mother, father)
        child2 = newChild(mother, father)
        if checkV(child1) and checkW(child1):
            childs.append(child1)
        if checkV(child2) and checkW(child2):
            childs.append(child2)
        return childs

In [8]:
# Мутация: добавление 1 случайной вещи 5% особей

def mutation(individual):
    flag = 1
    while flag:
        i = randint(0, len(individual)-1)
        if individual[i] == 0:
            individual[i] = 1
        flag = 0
    return individual

In [9]:
def GA():

    c = []
    C = 0
    for i in range (0, len(data)):
        c.append(data.at[i,'c'])
    for i in range (0, len(c)):
        C += c[i]
    
    minC = min(c)
    oldC = 0
    newC = C
    maxIteration = 500

    while maxIteration > 0 and abs(oldC - newC) > minC:
        oldC = newC

        # Начальная популяция
        population = createInitialPopulation()

        # Отбор
        population = selection(population)

        # Скрещивание
        childs = crossingover(population)

        # Мутация
        for i in range(0, round(len(population) * mutationInd)):
            rndIndividualInd = randint(0, len(population) - 1)
            population[rndIndividualInd] = mutation(population[rndIndividualInd])
        for i in range(0, round(len(childs) * mutationInd)):
            rndChildInd = randint(0, len(childs) - 1)
            childs[rndChildInd] = mutation(childs[rndChildInd])

        # Новая популяция
        for individual in population:
            individual.append(individualC(individual) * 0.9)
        for child in childs:
            child.append(individualC(child))
            population.extend(childs)
            population = list(filter(lambda x: checkW(x) and checkV(x), population))
            population.sort(key=lambda x: individualC(x), reverse=True)
            population=population[0:200]
            bestPopulation = population[0]

        newC = bestPopulation[-1]
        maxIteration -= 1

    return bestPopulation

In [10]:
bestPopulation = []
bestPopulation = GA()

In [11]:
sumW = 0
sumV = 0
sumC = 0
result = []
for i in range(numGenes):
    sumW += bestPopulation[i] * data.at[i,'w']
    sumV += bestPopulation[i] * data.at[i,'v']
    sumC += bestPopulation[i] * data.at[i,'c']
    if bestPopulation[i] == 1:
        result.append([data.at[i,'w'], data.at[i,'v'], data.at[i,'c']])
result_df = pd.DataFrame.from_records(result, columns=['w','v','c'])

In [12]:
print(result_df)
print("Вес:", sumW, "(Грузоподъемноcть:", W, ")")
print("Объем:", round(sumV,1), "(Вместимость:", V, ")")
print("Ценность:", sumC)

         w    v      c
0    796.0  1.0  158.0
1    439.0  0.7  176.0
2    595.0  0.7  316.0
3   1310.0  0.6  352.0
4    525.0  0.6  126.0
5   1126.0  0.6  377.0
6   1688.0  0.6  323.0
7    105.0  0.7  215.0
8   1550.0  0.6  339.0
9    261.0  0.9  287.0
10   478.0  0.9  257.0
11   651.0  0.6  167.0
12   132.0  1.1  319.0
13   873.0  1.1  339.0
14  1451.0  0.9  300.0
Вес: 11980.0 (Грузоподъемноcть: 13000 )
Объем: 11.6 (Вместимость: 12 )
Ценность: 4051.0


In [13]:
file=open('4_2.txt','w', encoding='utf-8')
file.write(result_df.to_string())
file.write("\nВес: " + str(sumW) + " (Грузоподъемноcть: " + str(W) + ")")
file.write("\nОбъем: " + str(round(sumV,1)) + " (Вместимость: " + str(V) + ")")
file.write("\nЦенность: " + str(sumC))
file.close()